In [1]:
# Read in PNC phenotype data

import pickle

fname = '../PNC_agesexwrat.pkl'

with open(fname, 'rb') as f:
    dct = pickle.load(f)

subs_age = list(dct['age'].keys())
subs_age = [str(subid) for subid in subs_age]
print(len(subs_age))

944


In [20]:
with open('../snps_age_927_small_correct.pkl', 'rb') as f:
    snps_feat = pickle.load(f)
    
print(len(snps_feat.keys()))

927


In [21]:
# Predict age - should be impossible with SNPs

import numpy as np

x = []
y = []
subs = []

for sub in dct['sex'].keys():
    substr = str(sub)
    if substr in snps_feat:
        xc = snps_feat[substr]
        x0 = xc == 0
        x1 = xc == 1
        x2 = xc == 2
        x.append(np.concatenate([x0,x1,x2]))
        y.append(dct['sex'][sub] == 'M')
        subs.append(sub)
        
x = np.stack(x)
y = np.array(y)

# nans = np.isnan(x)
# x[nans] = 0
# mu = np.mean(x, axis=0, keepdims=True)
# x = x - 1
# x[nans] = 0
        
print(len(x))
print(len(y))

927
927


In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

allrmse = []
task = 'sex'
bdir = f'../weights/snps/{task}'
subset = 'small'

def save(subset, i, w, trsubs, tsubs, desc):
    fname = f'{bdir}/{subset}{i}.pkl'
    dct = dict(w=w, trsubs=trsubs, tsubs=tsubs, desc=desc)
    with open(fname, 'wb') as f:
        pickle.dump(dct, f)

for i in range(20):
    idcs = np.random.permutation(len(x))
    ntrain = 700

    xtr = x[idcs[:ntrain]]
    xt = x[idcs[ntrain:]]

    ytr = y[idcs[:ntrain]]
    yt = y[idcs[ntrain:]]

#     mu = np.mean(ytr)
#     ytr = ytr - mu
#     yt = yt - mu

    clf = LogisticRegression(max_iter=1000).fit(xtr, ytr)
    yhat = clf.predict(xt)
    mat = confusion_matrix(yt, yhat, normalize='all')
    acc = np.sum(yhat == yt)/len(yt)
    print(mat)

#     rmse = np.mean((yhat-yt)**2)**0.5
    print(acc)
    
    save(subset, i, clf.coef_, [subs[i] for i in idcs[:ntrain]], [subs[i] for i in idcs[ntrain:]], 
         f'{task} prediction snps {subset} set cat(xRec,xHet,xDom) acc: {acc} mat: {mat}')
    
    allrmse.append(acc)
    
allrmse = np.array(allrmse)
print('---')
print(np.mean(allrmse))
print(np.std(allrmse))

[[0.55506608 0.        ]
 [0.         0.44493392]]
1.0
[[0.51101322 0.        ]
 [0.         0.48898678]]
1.0
[[0.52863436 0.        ]
 [0.         0.47136564]]
1.0
[[0.5154185 0.       ]
 [0.        0.4845815]]
1.0
[[0.5154185 0.       ]
 [0.        0.4845815]]
1.0
[[0.52422907 0.        ]
 [0.         0.47577093]]
1.0
[[0.53744493 0.        ]
 [0.         0.46255507]]
1.0
[[0.47577093 0.        ]
 [0.         0.52422907]]
1.0
[[0.55066079 0.        ]
 [0.         0.44933921]]
1.0
[[0.56387665 0.        ]
 [0.         0.43612335]]
1.0
[[0.55506608 0.        ]
 [0.         0.44493392]]
1.0
[[0.57709251 0.        ]
 [0.         0.42290749]]
1.0
[[0.57268722 0.        ]
 [0.         0.42731278]]
1.0
[[0.57709251 0.        ]
 [0.         0.42290749]]
1.0
[[0.54185022 0.        ]
 [0.         0.45814978]]
1.0
[[0.55947137 0.        ]
 [0.         0.44052863]]
1.0
[[0.55066079 0.        ]
 [0.         0.44933921]]
1.0
[[0.42731278 0.        ]
 [0.         0.57268722]]
1.0
[[0.54625551 0.   